In [1]:
import pandas as pd
import numpy as np
from scipy import stats 
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.api import OLS, add_constant
from statsmodels.stats.stattools import durbin_watson

In [2]:
# Load dataset
dataset = pd.read_excel("D:/Semester 5/Ekonometrika/tugas/data/Jumlah Wisata Nusantara.xlsx")
dataset = dataset.set_index(['Provinsi', 'Tahun'])
dataset

Jumlah_Wisnus  Daya_Tarik_Wisata_Alam  \
Provinsi       Tahun                                          
Aceh           2018         6518831                      10   
               2019        25523219                       7   
Sumatera Utara 2018        10345256                      48   
               2019        63576590                      46   
Sumatera Barat 2018         6402187                      37   
...                             ...                     ...   
Maluku Utara   2019         2288358                       3   
Papua Barat    2018          686836                       8   
               2019         2492235                       0   
Papua          2018         1354526                       8   
               2019         4551276                       6   

                      Daya_Tarik_Wisata_Budaya  Daya_Tarik_Wisata_Buatan  \
Provinsi       Tahun                                                       
Aceh           2018                          5                         3   
               2019                          5                         8   
Sumatera Utara 2018                         12                        14   
               2019                          8                        70   
Sumatera Barat 2018                         11                         4   
...                                        ...                       ...   
Maluku Utara   2019                          1                         0   
Papua Barat    2018                          0                         0   
               2019                          0                         2   
Papua          2018                          5                         1   
               2019                          1                         4   

                      Taman_Hiburan_dan_Rekreasi  Kawasan_Pariwisata  \
Provinsi       Tahun                                                   
Aceh           2018                           14                   0   
               2019                           14                   0   
Sumatera Utara 2018                           33                   2   
               2019                           24                   6   
Sumatera Barat 2018                           14                   4   
...                                          ...                 ...   
Maluku Utara   2019                            1                   1   
Papua Barat    2018                            0                   0   
               2019                            3                   7   
Papua          2018                            4                   0   
               2019                            5                   0   

                      Wisata_Tirta  
Provinsi       Tahun                
Aceh           2018             14  
               2019             12  
Sumatera Utara 2018             58  
               2019             16  
Sumatera Barat 2018              7  
...                            ...  
Maluku Utara   2019              0  
Papua Barat    2018              6  
               2019              2  
Papua          2018              3  
               2019              4  

[68 rows x 7 columns]

# Transformasi Data

In [3]:
# Transformasi log untuk variabel dependen dan independen
dataset_log = dataset.copy()
dataset_log['Jumlah_Wisnus'] = np.log(dataset['Jumlah_Wisnus'])

# Transformasi log untuk variabel independen (hanya jika relevan)
log_columns = ['Daya_Tarik_Wisata_Alam', 'Daya_Tarik_Wisata_Budaya', 
               'Daya_Tarik_Wisata_Buatan', 'Taman_Hiburan_dan_Rekreasi',
               'Kawasan_Pariwisata', 'Wisata_Tirta']
dataset_log[log_columns] = np.log(dataset[log_columns] + 1)

# Definisikan variabel dependen dan independen setelah transformasi
y_log = dataset_log['Jumlah_Wisnus']
X_log = dataset_log[log_columns]
X_log = add_constant(X_log)
dataset_log

Jumlah_Wisnus  Daya_Tarik_Wisata_Alam  \
Provinsi       Tahun                                          
Aceh           2018       15.690206                2.397895   
               2019       17.055099                2.079442   
Sumatera Utara 2018       16.152039                3.891820   
               2019       17.967756                3.850148   
Sumatera Barat 2018       15.672150                3.637586   
...                             ...                     ...   
Maluku Utara   2019       14.643345                1.386294   
Papua Barat    2018       13.439851                2.197225   
               2019       14.728690                0.000000   
Papua          2018       14.118962                2.197225   
               2019       15.330918                1.945910   

                      Daya_Tarik_Wisata_Budaya  Daya_Tarik_Wisata_Buatan  \
Provinsi       Tahun                                                       
Aceh           2018                   1.791759                  1.386294   
               2019                   1.791759                  2.197225   
Sumatera Utara 2018                   2.564949                  2.708050   
               2019                   2.197225                  4.262680   
Sumatera Barat 2018                   2.484907                  1.609438   
...                                        ...                       ...   
Maluku Utara   2019                   0.693147                  0.000000   
Papua Barat    2018                   0.000000                  0.000000   
               2019                   0.000000                  1.098612   
Papua          2018                   1.791759                  0.693147   
               2019                   0.693147                  1.609438   

                      Taman_Hiburan_dan_Rekreasi  Kawasan_Pariwisata  \
Provinsi       Tahun                                                   
Aceh           2018                     2.708050            0.000000   
               2019                     2.708050            0.000000   
Sumatera Utara 2018                     3.526361            1.098612   
               2019                     3.218876            1.945910   
Sumatera Barat 2018                     2.708050            1.609438   
...                                          ...                 ...   
Maluku Utara   2019                     0.693147            0.693147   
Papua Barat    2018                     0.000000            0.000000   
               2019                     1.386294            2.079442   
Papua          2018                     1.609438            0.000000   
               2019                     1.791759            0.000000   

                      Wisata_Tirta  
Provinsi       Tahun                
Aceh           2018       2.708050  
               2019       2.564949  
Sumatera Utara 2018       4.077537  
               2019       2.833213  
Sumatera Barat 2018       2.079442  
...                            ...  
Maluku Utara   2019       0.000000  
Papua Barat    2018       1.945910  
               2019       1.098612  
Papua          2018       1.386294  
               2019       1.609438  

[68 rows x 7 columns]

# Estimasi Model

In [4]:
# Common effects model dengan data log
common_model_log = PanelOLS(y_log, X_log)
common_res_log = common_model_log.fit()
print(common_res_log)

                          PanelOLS Estimation Summary                           
Dep. Variable:          Jumlah_Wisnus   R-squared:                        0.7134
Estimator:                   PanelOLS   R-squared (Between):              0.7525
No. Observations:                  68   R-squared (Within):               0.5563
Date:                Sat, Nov 16 2024   R-squared (Overall):              0.7134
Time:                        18:01:42   Log-likelihood                   -68.074
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      25.311
Entities:                          34   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(6,61)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             25.311
                            

In [5]:
# Fixed effects model dengan data log
fixed_model_log = PanelOLS(y_log, X_log, entity_effects=True)
fixed_res_log = fixed_model_log.fit()
print(fixed_res_log)

                          PanelOLS Estimation Summary                           
Dep. Variable:          Jumlah_Wisnus   R-squared:                        0.6934
Estimator:                   PanelOLS   R-squared (Between):              0.6044
No. Observations:                  68   R-squared (Within):               0.6934
Date:                Sat, Nov 16 2024   R-squared (Overall):              0.6221
Time:                        18:01:42   Log-likelihood                   -15.460
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      10.556
Entities:                          34   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(6,28)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             10.556
                            

In [6]:
# Random effects model dengan data log
random_model_log = RandomEffects(y_log, X_log)
random_res_log = random_model_log.fit()
print(random_res_log)

                        RandomEffects Estimation Summary                        
Dep. Variable:          Jumlah_Wisnus   R-squared:                        0.6871
Estimator:              RandomEffects   R-squared (Between):              0.7131
No. Observations:                  68   R-squared (Within):               0.6513
Date:                Sat, Nov 16 2024   R-squared (Overall):              0.7008
Time:                        18:01:42   Log-likelihood                   -45.573
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      22.322
Entities:                          34   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(6,61)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             22.322
                            

# Residual Common Effects (setelah di estimasi)

In [7]:
# Mendefinisikan residual dari model Common Effects
ols_residuals_log = common_res_log.resids
ols_residuals_log

Provinsi        Tahun
Aceh            2018     0.519978
                2019     1.428509
Sumatera Utara  2018    -0.249432
                2019     0.473301
Sumatera Barat  2018    -0.134745
                           ...   
Maluku Utara    2019     0.323361
Papua Barat     2018    -0.703750
                2019    -0.595145
Papua           2018    -0.551829
                2019     0.146696
Name: residual, Length: 68, dtype: float64

# Compare Model

In [8]:
comparison_log = compare({
    "Common Effects (Log)": common_res_log,
    "Fixed Effects (Log)": fixed_res_log,
    "Random Effects (Log)": random_res_log
})
print(comparison_log)

                                      Model Comparison                                      
                               Common Effects (Log) Fixed Effects (Log) Random Effects (Log)
--------------------------------------------------------------------------------------------
Dep. Variable                         Jumlah_Wisnus       Jumlah_Wisnus        Jumlah_Wisnus
Estimator                                  PanelOLS            PanelOLS        RandomEffects
No. Observations                                 68                  68                   68
Cov. Est.                                Unadjusted          Unadjusted           Unadjusted
R-squared                                    0.7134              0.6934               0.6871
R-Squared (Within)                           0.5563              0.6934               0.6513
R-Squared (Between)                          0.7525              0.6044               0.7131
R-Squared (Overall)                          0.7134              0.622

# Uji Lagrange Multiplier (Perbandingan Common Effects & Random Effects)

In [9]:
# Lagrange Multiplier Test
lm_stat = len(y_log) * (common_res_log.rsquared - random_res_log.rsquared)
p_value_lm = 1 - stats.chi2.cdf(lm_stat, df=1)

print(f"LM Statistic: {lm_stat}")
print(f"P-value: {p_value_lm}")

if p_value_lm < 0.05:
    print("Model Random Effects lebih sesuai (Common Effects ditolak).")
else:
    print("Model Common Effects lebih sesuai.")

LM Statistic: 1.7927827629025161
P-value: 0.18058747833614897
Model Common Effects lebih sesuai.


# Uji Hausman (Perbandingan Fixed Effects & Random Effects)

In [10]:
import numpy as np
from linearmodels.panel import RandomEffects
from scipy import stats

# Model Fixed Effects (PanelOLS dengan entity_effects=True)
fixed_effects_model = PanelOLS(y_log, X_log, entity_effects=True)
fixed_results = fixed_effects_model.fit()

# Model Random Effects (RandomEffects)
random_effects_model = RandomEffects(y_log, X_log)
random_results = random_effects_model.fit()

# Koefisien dan varians dari kedua model
fixed_coefficients = fixed_results.params
random_coefficients = random_results.params
fixed_cov_matrix = fixed_results.cov
random_cov_matrix = random_results.cov

# Perhitungan Hausman Test
# Perhitungan statistik uji Hausman
diff_coeff = fixed_coefficients - random_coefficients
diff_cov = fixed_cov_matrix + random_cov_matrix  # Varians gabungan
hausman_stat = np.dot(diff_coeff.T, np.linalg.inv(diff_cov).dot(diff_coeff))

# Derajat kebebasan (df) dan p-value
df = len(fixed_coefficients)  # Jumlah parameter
p_value = 1 - stats.chi2.cdf(hausman_stat, df)

# Menampilkan hasil Hausman Test
print("Hausman Statistic:", hausman_stat)
print("p-value:", p_value)

# Menentukan model yang lebih sesuai
if p_value < 0.05:
    print("Model Fixed Effects lebih tepat (p-value < 0.05)")
else:
    print("Model Random Effects lebih tepat (p-value >= 0.05)")

Hausman Statistic: 2.2546617243509344
p-value: 0.9444122488796046
Model Random Effects lebih tepat (p-value >= 0.05)


# Estimasi WLS (Random Effects Model)

In [11]:
# Weighted Least Squares (WLS)
fitted_values = common_res_log.fitted_values
absolute_residuals = abs(ols_residuals_log)
weights = 1 / OLS(absolute_residuals, add_constant(fitted_values)).fit().fittedvalues ** 2

# Re-estimate WLS model untuk Random Effects
wls_model = RandomEffects(y_log, X_log, weights=weights)
wls_res = wls_model.fit()
print("\nHasil WLS Model:")
print(wls_res)


Hasil WLS Model:
                        RandomEffects Estimation Summary                        
Dep. Variable:          Jumlah_Wisnus   R-squared:                        0.8746
Estimator:              RandomEffects   R-squared (Between):              0.7300
No. Observations:                  68   R-squared (Within):               0.6682
Date:                Sat, Nov 16 2024   R-squared (Overall):              0.7187
Time:                        18:01:42   Log-likelihood                   -43.265
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      167.35
Entities:                          34   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(6,61)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             25.137
          

# Uji Heteroskedastisitas dengan Uji Breusch-Pagan

In [12]:
# Mendefinisikan residual dari model WLS
wls_residuals = wls_res.resids

# Lakukan uji Breusch-Pagan pada residual dari model WLS
bp_test_wls = het_breuschpagan(wls_residuals, X_log)

# Tampilkan hasil uji Breusch-Pagan setelah WLS
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nUji Heteroskedastisitas (Breusch-Pagan Test) Setelah WLS:")
print(dict(zip(labels, bp_test_wls)))


Uji Heteroskedastisitas (Breusch-Pagan Test) Setelah WLS:
{'LM Statistic': 9.56149982008438, 'LM-Test p-value': 0.14437453157049027, 'F-Statistic': 1.6634338869908267, 'F-Test p-value': 0.14543099997889974}


# Uji Autokorelasi dengan Uji Durbin-Watson

In [13]:
# Durbin-Watson Test
dw_test = durbin_watson(ols_residuals_log)
print("Durbin-Watson Test:", dw_test)

Durbin-Watson Test: 1.735682739532569


# Uji Multikolinearitas dengan VIF

In [14]:
# Variance Inflation Factor (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X_log.columns
vif_data["VIF"] = [variance_inflation_factor(X_log.values, i) for i in range(X_log.shape[1])]
print(vif_data)

                      feature       VIF
0                       const  6.510688
1      Daya_Tarik_Wisata_Alam  4.107600
2    Daya_Tarik_Wisata_Budaya  2.568511
3    Daya_Tarik_Wisata_Buatan  2.417400
4  Taman_Hiburan_dan_Rekreasi  3.237801
5          Kawasan_Pariwisata  2.457758
6                Wisata_Tirta  2.915864


# Evaluasi Signifikansi Variabel dalam Model Random Effects (after WLS)

In [15]:
# Menampilkan koefisien dan p-value untuk model Random Effects WLS
print("\nPengaruh Semua Variabel dalam Model Random Effects WLS:")
print(wls_res.params)  # Koefisien setiap variabel
print(wls_res.pvalues)  # P-value untuk menguji signifikansi koefisien

# Menampilkan informasi koefisien beserta P-value dan CI
conf_int = wls_res.conf_int()  # Mendapatkan DataFrame dengan kolom [lower, upper]
summary = pd.DataFrame({
    'Coefficient': wls_res.params,
    'P-value': wls_res.pvalues,
    'Lower CI': conf_int.iloc[:, 0],  # Kolom pertama (lower bound)
    'Upper CI': conf_int.iloc[:, 1],  # Kolom kedua (upper bound)
})

print("\nSummary Koefisien dan P-value:")
print(summary)


Pengaruh Semua Variabel dalam Model Random Effects WLS:
const                         14.176126
Daya_Tarik_Wisata_Alam         0.154229
Daya_Tarik_Wisata_Budaya      -0.053357
Daya_Tarik_Wisata_Buatan       0.655157
Taman_Hiburan_dan_Rekreasi    -0.108940
Kawasan_Pariwisata             0.396524
Wisata_Tirta                  -0.076522
Name: parameter, dtype: float64
const                         0.000000e+00
Daya_Tarik_Wisata_Alam        3.170998e-01
Daya_Tarik_Wisata_Budaya      7.306579e-01
Daya_Tarik_Wisata_Buatan      1.409690e-09
Taman_Hiburan_dan_Rekreasi    4.569438e-01
Kawasan_Pariwisata            5.499700e-03
Wisata_Tirta                  4.488314e-01
Name: pvalue, dtype: float64

Summary Koefisien dan P-value:
                            Coefficient       P-value   Lower CI   Upper CI
const                         14.176126  0.000000e+00  13.666867  14.685386
Daya_Tarik_Wisata_Alam         0.154229  3.170998e-01  -0.151509   0.459968
Daya_Tarik_Wisata_Budaya      -0.053357  